# Why Agents?

What is an agent?

An agent is a reference to combining models that can perform some kind of reasoning, like large language models (e.g ChatGPT, Llama2, Mistral, etc...) with tools to give it access to the real world,
so they can do things like browsing the internet, buying stuff, etc...

Ok, so, why is there so much hype around agents right now?

Because Agents are cool! Recently with the advance of LLMs, we've seen them become an amazing tool to do all sorts of things like building apps, browse the internet and more.



SOme neat examples of these kinds of agents can be found in here:

- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)

Now, today although they seem extremely powerful, agents are still at a very early stage in terms of readiness to deploy as products, something you can atest by listening to Andrej Karpathy talk about agents in this talk here:

- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)

This live-training is all about! Getting you excited about this amazing new technology, understanding it from the ground up but with a focus on practical applications and fun stuff you can do with them! 

# What is an Agent?

An agent is nothing more than some entity that can _think_ and _act_, that's right, in a way you're an agent! 

After all you can think and act on those thoughts like in the case of coming to this live-training:

- Thought: "I want to learn about agents"

- Action: "Go to the internet and research cool platforms where I can learn about agents"

- Thought: "O'Reilly has some awesome courses and live-trainings"

- Action: "Look up O'Reilly courses"

- Thought: "Live-trainings by instructor Lucas are awesome"

- Action: "Schedule live-training about agents with instructor Lucas Soares" (lol)

In a way this is a simplified rendition of what brought you here, obviously not necessarily in this particular order nor these particular sets of thought and action pairs. This particular way of thinking about how to structure thoughts and actions is well represented in the paper: [ReACT](https://arxiv.org/pdf/2210.03629.pdf). 

With regards to LLMs, how can bring this idea to fruition thinking about the LLM model as the reasoning and thinking engine?

We can start simple and just call the openai API to start:

In [1]:
%pip install openai==2.6.1

Note: you may need to restart the kernel to use updated packages.


In [24]:
# uncomment this if running locally
#!pip install python-dotenv
# from dotenv import load_dotenv

# load_dotenv()

import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [25]:
from openai import OpenAI
from IPython.display import Markdown

client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            # The control over the behavior of the model
            {"role": "system", "content": "You are a helpful research and programming assistant"},
            # The prompt from the user
            {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

output = get_response("Create a joke about a bald teacher explaining agents to wonderful students.")
Markdown(output)

Why did the bald teacher love explaining agents to his wonderful students?

Because he knew that sometimes the best ideas come from "shedding" old beliefs!

![](2025-10-28-14-22-39.png)

Ok cool, so here we have three ideas of actions to perform:

- Creating directories
- Listing files
- Removing files

Let's transform them into functions that we could call just like in any type of Python-based application.

In [26]:
def create_directory(dir_name):
    os.makedirs(dir_name, exist_ok=True)

def create_file(filename):
    with open(filename, 'w'):
        pass

def list_files():
    files = os.listdir()
    for file in files:
        print(file)

Now, let's imagine that we wanted to create an agent that would perform these actions for us based on some input that we give it, how can we connect models that we know and can use today like ChatGPT, with these tools that do stuff in the real world?

To answer this question, how about we give a task to the model, and for that task we ask it to list the steps that it needs to perform to complete the task, and then for each of those steps we would ask the model to decide whether or not a function should be called to execute that task? 

In the famous paper ['Toolformer'](https://arxiv.org/pdf/2302.04761.pdf) they demonstrated that today's advanced LLMs like the gpt-series could teacha themselves how to properly call and use external tools!

Isn't that awesome???

So, let's see if we can hack our way into connecting the llm response with the functions that we want that llm to use.

<img src="./assets-resources/2025-10-28-14-25-05.png" width=40%>

Now, how can we actually put it all together so that given a task, a model can:

- Plan the task
- Execute actions to complete the task
- Know when to call a function


This is actually an interesting problem, let's understand why is that the case by trying to hack our way into putting all of these together:

In [27]:
def get_response(prompt_question, model="gpt-4o-mini"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

def create_directory(dir_name):
    os.makedirs(dir_name, exist_ok=True)

def create_file(filename):
    with open(filename, 'w'):
        pass

def list_files():
    files = os.listdir()
    for file in files:
        print(file)

    

task_description = "Create a folder called 'funny-pancakes-recipes'. Inside that folder, \
create a file called '3-funny-pancake-recipes.md"

prompt = f"""Given this task: {task_description}, \n
        Consider you have access to the following functions:
                            
    def create_directory(dir_name):
        os.makedirs(dir_name, exist_ok=True)

    def create_file():
        with open('test.txt', 'w'):
            pass

    def list_files():
        files = os.listdir()
        for file in files:
            print(file)
    
    Your output should be the first function to be executed to complete the task containing the necessary arguments.
    The OUTPUT SHOULD ONLY BE THE PYTHON FUNCTION CALL and NOTHING ELSE.
    """

output = get_response(prompt)

Markdown(output)

```python
create_directory('funny-pancakes-recipes')
```

In [28]:
import os
# Remove any ```python or ``` tags from the output string if present
output_clean = output.replace("```python", "").replace("```", "").strip()
Markdown(output_clean)

create_directory('funny-pancakes-recipes')

In [29]:
exec(output_clean)

In [30]:
!ls -d ./* | grep pancakes

./funny-pancakes-recipes
./pancakes-are-better-than-waffles


![](2025-10-28-14-32-16.png)

At this point we can start identifying a lot of issues with this approach despite our early sucess:

- Uncertainty of model's outputs can affect our ability to reliably call the functions
- We need more structured ways to prepare the inputs of the function calls
- We need better ways to put everything together (just feeding the entire functions like this makes it a very clunky and non-scalable framework for more complex cases)

There are many more issues but starting with these, we can now look at frameworks and see how they fix these issues and with that in mind understand what is behind their implementations!

I personally think this is a much better way to understand what is going on behind agents in practice rather than just use the more higher level frameworks right of the bat!

# OpenAI Functions

Ok, let's first understand how [OpenAI](https://openai.com/) the company behind ChatGPT, allows for these function call implementations in its API.

OpenAI implemented a [function calling API](https://platform.openai.com/docs/guides/function-calling) which is a standard way to connect their models to outside tools like in the very simple example we did above.

According to their [official documentation](https://platform.openai.com/docs/guides/function-calling#:~:text=The%20basic%20sequence,to%20the%20user.) the sequence of steps for function calling is as follows:
1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

Below is an example taken from their official documentation:

In [10]:
from openai import OpenAI
import json

client = OpenAI()

Let's look at how our previous model with those three simple functions: `create_directory()`, `create_file()`, and `list_files()` would be implemented using OpenAI's function calling approach:

In [32]:
import json
import subprocess
import json
import os

def create_directory(directory_name):
    """Function that creates a directory given a directory name."""
    os.mkdir(directory_name)
    return json.dumps({"directory_name": directory_name})


tool_create_directory = {
    "type": "function",
    "function": {
        "name": "create_directory",
        "description": "Create a directory given a directory name.",
        "parameters": {
            "type": "object",
            "properties": {
                "directory_name": {
                    "type": "string",
                    "description": "The name of the directory to create.",
                }
            },
            "required": ["directory_name"],
        },
    },
}

tools = [tool_create_directory]

In [34]:
import json

def run_terminal_task():
    messages = [
        {"role": "user", 
         "content": "Create a folder called 'pancakes-are-better-than-waffles'."}]
    tools = [tool_create_directory]  
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "create_directory": create_directory,
        }
        messages.append(response_message)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                directory_name=function_args.get("directory_name"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return second_response

output = run_terminal_task()
output

ChatCompletion(id='chatcmpl-CVf91lUDyYC4BlqEbk87d5H01yKI6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="A folder named 'pancakes-are-better-than-waffles' has been created successfully.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1761662363, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=21, prompt_tokens=74, total_tokens=95, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [35]:
output.choices[0].message.content

"A folder named 'pancakes-are-better-than-waffles' has been created successfully."

In [36]:
!ls -d */ | grep waffles

pancakes-are-better-than-waffles/


Great! We implemented openai function calling for creating directories! We could evolve this approach but let's stop for now.

See more info on these examples from OpenAI's [official cookbook](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models).

In [15]:
import os

def create_dir(folder_path):
    """
    Creates a directory given a folder path.
    """
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    
    return f"Folder path was created at: {folder_path}"


create_dir("lucas-test-agent")

'Folder path was created at: lucas-test-agent'

In [16]:
!ls -d */ | grep lucas-test-agent

lucas-test-agent/


In [37]:
def create_file(file_path, contents=""):
    """
    Creates a file with content.
    If no content is provided it will create an empty file.
    """
    
    with open(file_path, "w") as f:
        f.write(contents)
    
    return f"A file was created at: {file_path}"

create_file("./lucas-test-agent/file-text.txt", "This is a test")

'A file was created at: ./lucas-test-agent/file-text.txt'

In [38]:
def read_file(file_path):
    """
    Reads from file given its path.
    """
    
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

read_file("./lucas-test-agent/file-text.txt")

'This is a test'

In [19]:
# Define tools in OpenAI format
tools = [
    {
        "type": "function",
        "function": {
            "name": "create_dir",
            "description": "Creates a directory given a folder path.",
            "parameters": {
                "type": "object",
                "properties": {
                    "folder_path": {
                        "type": "string",
                        "description": "The path of the folder to create"
                    }
                },
                "required": ["folder_path"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_file",
            "description": "Creates a file with content. If no content is provided it will create an empty file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "The path where the file should be created"
                    },
                    "contents": {
                        "type": "string",
                        "description": "The content to write to the file (optional)"
                    }
                },
                "required": ["file_path"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads from file given its path.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "The path of the file to read"
                    }
                },
                "required": ["file_path"]
            }
        }
    }
]

# Create mapping of function names to actual functions
tool_mapping = {
    "create_dir": create_dir,
    "create_file": create_file,
    "read_file": read_file
}

def llm_call(query, observations=[], actions_taken=[]):
    """
    Calls the llm using OpenAI's chat completions API with function calling.
    It can return a tool call with arguments for calling different tools,
    or an output to the user.
    """
    system_prompt = """
    Imagine you are a simple assistant tasked with managing a file system. You have access to three tools:
    
    1. create_dir: Creates a new directory.
    2. create_file: Creates a new file and optionally writes content to it.
    3. read_file: Reads the contents of an existing file.

    Based on the user input and your observations, choose an action to execute. Your action must follow these guidelines:
    
    * Action Guidelines *
    1) Only one action is allowed per iteration.
    2) Be concise and specific about what to create, write, or read.
    3) Provide clear reasoning for your action.
    4) Always consider the current context before taking action.
    """

    # Prepare messages
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"""
User Query: {query}
Observations: {observations}
Actions Taken So Far: {actions_taken}
"""}
    ]

    # Call OpenAI API
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    response_message = response.choices[0].message
    
    # Check if the model wants to call a function
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            # Check if function_name exists in tool_mapping
            if function_name not in tool_mapping:
                print(f"Error: Unknown function name '{function_name}'")
                continue

            try:
                # Attempt to call the function with the provided arguments
                tool_output = tool_mapping[function_name](**function_args)
                actions_taken.append(function_name)
                observations.append(tool_output)
            except TypeError as e:
                # Handle errors if the number of arguments does not match
                print(f"Error: Invalid arguments for function '{function_name}': {e}")
    
    return response_message, actions_taken, observations

![](2025-10-28-14-49-49.png)

In [20]:
# Test 1: Create a directory
output, actions, observations = llm_call("Create a folder named: openai-tests-tool-calling")
print("Output:", output)
print("Actions:", actions)
print("Observations:", observations)

Output: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_NrwUV8lID7mZrQunXSjyGjDR', function=Function(arguments='{"folder_path":"openai-tests-tool-calling"}', name='create_dir'), type='function')])
Actions: ['create_dir']
Observations: ['Folder path was created at: openai-tests-tool-calling']


In [21]:
# Test 2: Create a file with content
output, actions, observations = llm_call(
    "Create a file at: ./openai-tests-tool-calling/test1.txt with the contents: Hello! This is a test using OpenAI's function calling API!",
    observations=["Folder path was created at: openai-tests-tool-calling"],
    actions_taken=["create_dir"]
)
print("Output:", output)
print("Actions:", actions)
print("Observations:", observations)

Output: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_2rrR7q1b8DosjOVsJcCyj8jq', function=Function(arguments='{"file_path":"./openai-tests-tool-calling/test1.txt","contents":"Hello! This is a test using OpenAI\'s function calling API!"}', name='create_file'), type='function')])
Actions: ['create_dir', 'create_file']
Observations: ['Folder path was created at: openai-tests-tool-calling', 'A file was created at: ./openai-tests-tool-calling/test1.txt']


In [22]:
# Test 3: Read file contents
output, actions, observations = llm_call(
    "Read the file contents from ./openai-tests-tool-calling/test1.txt",
    observations=[
        "Folder path was created at: openai-tests-tool-calling",
        "A file was created at: ./openai-tests-tool-calling/test1.txt"
    ],
    actions_taken=["create_dir", "create_file"]
)
print("Output:", output)
print("Actions:", actions)
print("Observations:", observations)

Output: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_nNs2UtkXuY9kjmV1uKkHE6NI', function=Function(arguments='{"file_path":"./openai-tests-tool-calling/test1.txt"}', name='read_file'), type='function')])
Actions: ['create_dir', 'create_file', 'read_file']
Observations: ['Folder path was created at: openai-tests-tool-calling', 'A file was created at: ./openai-tests-tool-calling/test1.txt', "Hello! This is a test using OpenAI's function calling API!"]


In [23]:
def agent_loop(query, max_iterations=5):
    """
    Agent loop that continues calling llm_call until the task is complete
    or max iterations is reached.
    """
    iter_count = 0
    obs = []
    acts_taken = []
    
    while True:
        output, acts_taken, obs = llm_call(query, obs, acts_taken)
        print(f"Iteration {iter_count + 1}:")
        print(f"  Tool calls: {output.tool_calls if output.tool_calls else 'None'}")
        print(f"  Content: {output.content if output.content else 'No content'}")
        print(f"  Actions taken: {acts_taken}")
        print(f"  Observations: {obs}")
        print()
        
        iter_count += 1
        
        # Break if we hit max iterations
        if iter_count >= max_iterations:
            print(f"Breaking after {iter_count} iterations")
            break
        
        # Break if the model returns content without tool calls (task complete)
        if output.content and not output.tool_calls:
            break
    
    return output.content if output.content else "Task completed"


# Test the agent loop
result = agent_loop("Create a folder in current directory named 'testing-openai-agents' and inside that folder create a file named test-file.txt with content 'Hello from OpenAI API!'")
print("\nFinal result:", result)

Iteration 1:
  Tool calls: [ChatCompletionMessageFunctionToolCall(id='call_nxVo8ehW5EVUIo0PicG7JhcR', function=Function(arguments='{"folder_path": "testing-openai-agents"}', name='create_dir'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_cYakPce7mem6VaiUH0C1lkB8', function=Function(arguments='{"file_path": "testing-openai-agents/test-file.txt", "contents": "Hello from OpenAI API!"}', name='create_file'), type='function')]
  Content: No content
  Actions taken: ['create_dir', 'create_file']
  Observations: ['Folder path was created at: testing-openai-agents', 'A file was created at: testing-openai-agents/test-file.txt']

Iteration 2:
  Tool calls: [ChatCompletionMessageFunctionToolCall(id='call_8flQU2Dpt92PhBxsXal4Pz4C', function=Function(arguments='{"folder_path": "testing-openai-agents"}', name='create_dir'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_SEbPtL4rZezBpBEcA8uoVXeE', function=Function(arguments='{"file_path": "testing-openai-agents

# References

- [HuggingGPT](https://github.com/microsoft/JARVIS)
- [Gen Agents](https://arxiv.org/pdf/2304.03442.pdf)
- [WebGPT](https://www.semanticscholar.org/paper/WebGPT%3A-Browser-assisted-question-answering-with-Nakano-Hilton/2f3efe44083af91cef562c1a3451eee2f8601d22)
- [LangChain](https://python.langchain.com/docs/get_started/introduction)
- [OpenAI](https://openai.com/)
- [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)
- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)
- [ReACT Paper](https://arxiv.org/abs/2210.03629)